<a href="https://www.kaggle.com/code/gamalelansary/ambulance-detector?scriptVersionId=280090580" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [46]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ambulance-dataset/carsdataset',
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='training',        
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ambulance-dataset/carsdataset',
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='validation', 
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=42                   
)


Found 4749 files belonging to 2 classes.
Using 3800 files for training.
Found 4749 files belonging to 2 classes.
Using 949 files for validation.


In [47]:
base_model=MobileNetV2(
    weights="imagenet",
    input_shape=(256,256,3),
    include_top=False
)
base_model.trainable=False
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255,input_shape=(256,256,3)),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2)
])

/tmp/ipykernel_48/2550062916.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model=MobileNetV2(


In [48]:
inputs=keras.Input(shape=(256,256,3))
x=data_augmentation(inputs)
x=base_model(x, training=False)
x=keras.layers.GlobalAveragePooling2D()(x)
x=keras.layers.Dense(786,activation="relu")(x)
outputs=keras.layers.Dense(2,activation="softmax")(x)
model=keras.Model(inputs,outputs)

In [49]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,batch_size=64,epochs=3)

Epoch 1/3
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 139ms/step - accuracy: 0.8655 - loss: 0.2976 - val_accuracy: 0.9968 - val_loss: 0.0104
Epoch 2/3
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 112ms/step - accuracy: 0.9818 - loss: 0.0426 - val_accuracy: 0.9958 - val_loss: 0.0142
Epoch 3/3
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 112ms/step - accuracy: 0.9870 - loss: 0.0343 - val_accuracy: 0.9968 - val_loss: 0.0119


In [51]:
base_model.trainable=True
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,batch_size=64,epochs=1)

60/60 ━━━━━━━━━━━━━━━━━━━━ 60s 466ms/step - accuracy: 0.9801 - loss: 0.0550 - val_accuracy: 0.9968 - val_loss: 0.0098


In [44]:
model.save("ambulance_detector.keras")

In [52]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/input/testing",
    labels=None,              
    shuffle=False,            
    image_size=(256, 256),               
)
predictions = model.predict(test_ds)
for image in predictions:
    if image[0]>image[1]:
        print(f"Ambulance with {image[0]*100}%")
    else:
        print(f"Not Ambulance with {image[1]*100}%")

Found 3 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Ambulance with 99.90234375%
Not Ambulance with 100.0%
Ambulance with 100.0%
